In [ ]:
# Step 1: Import Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from keras.models import Sequential
from keras.layers import Dense, GRU
from keras.optimizers import Adam
from sklearn.model_selection import train_test_split
import math

# Step 2: Load and Inspect the Data
file_path = 'Bangalore_1990_2022_BangaloreCity.csv'  # Update this path if needed
df = pd.read_csv(file_path)

# Display the first few rows to understand the data
print("Data Sample:")
print(df.head())

# Step 3: Data Preprocessing
# We will use the 'tavg' column for average temperature prediction
df = df.dropna(subset=['tavg'])  # Remove rows where 'tavg' is NaN
df['time'] = pd.to_datetime(df['time'], format='%d-%m-%Y')
df.set_index('time', inplace=True)

# Visualize the temperature data
plt.figure(figsize=(12, 6))
plt.plot(df['tavg'], label='Average Temperature over time')
plt.xlabel('Date')
plt.ylabel('Temperature (°C)')
plt.title('Average Temperature Trend in Bangalore')
plt.legend()
plt.show()

# Step 4: Prepare the Data for GRU
# Normalize the temperature data for better performance
scaler = MinMaxScaler(feature_range=(0, 1))
df['tavg'] = scaler.fit_transform(df[['tavg']])

# Convert data into a supervised learning problem
def create_dataset(data, look_back=30):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back), 0])
        y.append(data[i + look_back, 0])
    return np.array(X), np.array(y)

look_back = 30  # Number of previous time steps to consider
data = df['tavg'].values.reshape(-1, 1)
X, y = create_dataset(data, look_back)

# Reshape the input to be [samples, time steps, features] for GRU input
X = np.reshape(X, (X.shape[0], X.shape[1], 1))

# Step 5: Split the Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Step 6: Build the GRU Model with Updated Hyperparameters
model = Sequential()

# First GRU layer with 240 units and ReLU activation
model.add(GRU(240, activation='relu', input_shape=(look_back, 1), return_sequences=True))

# Second GRU layer with 240 units and ELU activation
model.add(GRU(240, activation='elu'))

# Dense output layer
model.add(Dense(1))

# Compile the model with updated learning rate and loss function
model.compile(optimizer=Adam(learning_rate=0.00015), loss='mean_squared_error')

# Step 7: Train the Model
epochs = 20  # Updated to match the specified hyperparameters
history = model.fit(X_train, y_train, epochs=epochs, batch_size=32, validation_data=(X_test, y_test), verbose=1)

# Step 8: Evaluate the Model
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.title('Model Loss')
plt.legend()
plt.show()

# Step 9: Make Predictions and Inverse Transform to Actual Scale
train_predict = model.predict(X_train)
test_predict = model.predict(X_test)

train_predict = scaler.inverse_transform(train_predict)
y_train_inv = scaler.inverse_transform(y_train.reshape(-1, 1))
test_predict = scaler.inverse_transform(test_predict)
y_test_inv = scaler.inverse_transform(y_test.reshape(-1, 1))

# Step 10: Calculate Accuracy and Plot Results
train_score = math.sqrt(mean_squared_error(y_train_inv, train_predict))
test_score = math.sqrt(mean_squared_error(y_test_inv, test_predict))
print(f"Train RMSE: {train_score:.2f}")
print(f"Test RMSE: {test_score:.2f}")

plt.figure(figsize=(12, 6))
plt.plot(y_test_inv, label='Actual Temperature')
plt.plot(test_predict, label='Predicted Temperature')
plt.xlabel('Time')
plt.ylabel('Temperature (°C)')
plt.title('Temperature Prediction vs Actual')
plt.legend()
plt.show()

# Calculate and display Mean Absolute Error (MAE)
train_mae = mean_absolute_error(y_train_inv, train_predict)
test_mae = mean_absolute_error(y_test_inv, test_predict)
print(f"Train MAE: {train_mae:.2f}")
print(f"Test MAE: {test_mae:.2f}")

# Step 11: Calculate R-squared and Accuracy Percentage
train_r2 = r2_score(y_train_inv, train_predict)
test_r2 = r2_score(y_test_inv, test_predict)
train_accuracy = train_r2 * 100
test_accuracy = test_r2 * 100
print(f"Train R² Score: {train_r2:.2f}")
print(f"Test R² Score: {test_r2:.2f}")
print(f"Train Accuracy: {train_accuracy:.2f}%")
print(f"Test Accuracy: {test_accuracy:.2f}%")

# Step 12: Conclusion and Analysis
print("The GRU model provides a reasonable prediction of future temperatures.")